
# Importing basic libraries for data analysis

In [10]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
sns.set_theme()

# Setting the best style for the plots in seaborn

import datetime as dt
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [11]:
# Reading the data from the csv file
df = pd.read_csv('airline_df.csv')
df.head()

,Airline,Country,Review,Date_Published,Type of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value for Money,Recommended
0,indigo-airlines,India,✅ Trip Verified | Flight was punctual. But no ...,2023-05-10,Solo Leisure,Economy Class,Abu Dhabi to Kochi,1.0,1.0,1.0,NaN,3.0,1.0,no
1,indigo-airlines,India,"✅ Trip Verified | My sister, niece and mother...",2023-05-07,Family Leisure,Economy Class,Mumbai to Mangalore,1.0,NaN,NaN,NaN,1.0,1.0,no
2,indigo-airlines,India,✅ Trip Verified | My 77-year-old father was fl...,2023-04-28,Solo Leisure,Economy Class,Abu Dhabi to Kochi,2.0,2.0,NaN,NaN,2.0,4.0,no
3,indigo-airlines,India,Not Verified | IndiGo are a low cost airline ...,2023-04-24,Solo Leisure,Economy Class,Jaipur to Ahmedabad,2.0,2.0,NaN,NaN,2.0,3.0,yes
4,indigo-airlines,India,✅ Trip Verified | My flight 6e 1176 which was...,2023-04-22,Family Leisure,Economy Class,Colombo to Mumbai via Chennai,1.0,1.0,1.0,1.0,1.0,1.0,no


# Data Description

In [12]:
# Description of the data
df.describe(include='all')

,Airline,Country,Review,Date_Published,Type of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value for Money,Recommended
count,15220,15220,15220,15220,10974,15047,10962,14222.000000,14208.000000,13085.000000,11762.000000,10720.000000,15044.000000,15220
unique,19,9,15205,3718,4,4,6716,NaN,NaN,NaN,NaN,NaN,NaN,2
top,emirates,India,Manchester to Doha and then Bangkok 24th Janua...,2015-01-14,Solo Leisure,Economy Class,Guangzhou to Sydney,NaN,NaN,NaN,NaN,NaN,NaN,yes
freq,2266,3640,2,39,4534,11214,30,NaN,NaN,NaN,NaN,NaN,NaN,8913
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.425116,3.588401,3.343676,3.432409,3.213433,3.372108,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.390692,1.524603,1.439589,1.408876,1.637206,1.533749,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,NaN


In [13]:
# Info of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15220 entries, 0 to 15219
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Airline                 15220 non-null  object 
 1   Country                 15220 non-null  object 
 2   Review                  15220 non-null  object 
 3   Date_Published          15220 non-null  object 
 4   Type of Traveller       10974 non-null  object 
 5   Seat Type               15047 non-null  object 
 6   Route                   10962 non-null  object 
 7   Seat Comfort            14222 non-null  float64
 8   Cabin Staff Service     14208 non-null  float64
 9   Food & Beverages        13085 non-null  float64
 10  Inflight Entertainment  11762 non-null  float64
 11  Ground Service          10720 non-null  float64
 12  Value for Money         15044 non-null  float64
 13  Recommended             15220 non-null  object 
dtypes: float64(6), object(8)
memory usage:

In [14]:
# Checking the null values in the data
df.isnull().sum().sort_values(ascending=False)

Ground Service            4500
Route                     4258
Type of Traveller         4246
Inflight Entertainment    3458
Food & Beverages          2135
Cabin Staff Service       1012
Seat Comfort               998
Value for Money            176
Seat Type                  173
Airline                      0
Country                      0
Review                       0
Date_Published               0
Recommended                  0
dtype: int64

# Data Cleaning and Preprocessing

* ### Data Imputation

In [15]:
# Separating the numerical and categorical columns
num_col = df.select_dtypes(include=np.number).columns.tolist()
obj_col = df.select_dtypes(include='object').columns.tolist()
cat_col = [x for x in obj_col if df[x].nunique() < 21]

print('Numerical Columns: ',num_col)
print('Categorical Columns: ',cat_col)

Numerical Columns:  ['Seat Comfort', 'Cabin Staff Service', 'Food & Beverages', 'Inflight Entertainment', 'Ground Service', 'Value for Money']
Categorical Columns:  ['Airline', 'Country', 'Type of Traveller', 'Seat Type', 'Recommended']


In [16]:
# Imputing the missing values in the numerical columns with mode using simple imputer
from sklearn.impute import KNNImputer

def cleanse_data(df):    
    
    imputer = KNNImputer(n_neighbors=7)
    df[num_col] = imputer.fit_transform(df[num_col])

    # As well as changing the data type of the numerical columns to int8
    
    df[num_col] = df[num_col].astype('int8')
    
    # Changing the Date_Published column to datetime format

    df['Date_Published'] = pd.to_datetime(df['Date_Published'])
    
    # Sort the data by Airline and Date_Published so that we can fill the missing values in the categorical columns

    df.sort_values(by=['Airline','Date_Published'],inplace=True)
    
    # Imputing the missing values in the categorical columns with mode using Backward fill
    
    df[obj_col] = df[obj_col].fillna(method='bfill')

    # Mapping Recommended to 1 and Not Recommended to 0
    
    df['Recommended'] = df['Recommended'].map({'yes':1,'no':0})

    # As well as changing the data type of the categorical columns to category
    
    df[cat_col] = df[cat_col].astype('category')

    # Creating a new Column Overall Rating by taking the mean of all the ratings given by the user upto 2 decimal places

    df['Overall_Rating'] = df[num_col].mean(axis=1).round().astype('float32')
    df['Route'] = df['Route'].apply(lambda x: x.split('via')[0].strip() if 'via' in x else x.strip())

    # Dropping the record where the route is Melbourne, Chennai and Zurich
    
    df.drop(df[df['Route'] == 'Melbourne'].index, inplace=True)
    df.drop(df[df['Route'] == 'Chennai'].index, inplace=True)
    df.drop(df[df['Route'] == 'Zurich'].index, inplace=True)

    # Small casing the Route column

    df['Route'] = df['Route'].str.lower()
    
    # ro, to, yo, - will be used for splitting the route column

    # Splitting the Route column into Origin and Destination

    # Origin Extraction

    df['Origin'] = df['Route'].apply(lambda x: x.split('ro')[0].strip() if 'ro' in x else x.strip())
    df['Origin'] = df['Origin'].apply(lambda x: x.split('to')[0].strip() if 'to' in x else x.strip())
    df['Origin'] = df['Origin'].apply(lambda x: x.split('yo')[0].strip() if 'yo' in x else x.strip())
    df['Origin'] = df['Origin'].apply(lambda x: x.split('-')[0].strip() if '-' in x else x.strip())
    df['Origin'] = df['Origin'].str.capitalize()

    # Destination Extraction

    df['Destination'] = df['Route'].apply(lambda x: x.split('ro')[-1].strip() if 'ro' in x else x.strip())
    df['Destination'] = df['Destination'].apply(lambda x: x.split('to')[-1].strip() if 'to' in x else x.strip())
    df['Destination'] = df['Destination'].apply(lambda x: x.split('yo')[-1].strip() if 'yo' in x else x.strip())
    df['Destination'] = df['Destination'].apply(lambda x: x.split('-')[-1].strip() if '-' in x else x.strip())
    df['Destination'] = df['Destination'].str.capitalize()

    # lastly dropping the
    
    df.drop_duplicates(inplace=True)

    # Preprocessing the Airline column

    df['Airline'] = df['Airline'].apply(lambda x: re.sub(r'-',' ',x))
    df['Airline'] = df['Airline'].str.capitalize()

    df.drop(['Route'],axis=1,inplace=True)
    
    df.reset_index(drop=True,inplace=True)
    
    return df

df = cleanse_data(df)

In [18]:
df[df['Overall_Rating'] == 3.0].head(50)

,Airline,Country,Review,Date_Published,Type of Traveller,Seat Type,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value for Money,Recommended,Overall_Rating,Origin,Destination
3,Air china,China,London to Sydney return via Beijing. A cheap f...,2012-02-22,Solo Leisure,Economy Class,4,1,2,4,3,4,0,3.0,Beijing,Xi'an
11,Air china,China,SFO-PEK in economy. Rebooked on CA after UA de...,2012-04-10,Solo Leisure,Economy Class,3,4,2,2,2,3,0,3.0,Beijing,Xi'an
19,Air china,China,Paris to Beijing in business class on a 777-30...,2012-05-29,Solo Leisure,Business Class,4,4,1,2,4,4,1,3.0,Beijing,Xi'an
21,Air china,China,Frankfurt to Taipei return. On the way to Taip...,2012-06-17,Solo Leisure,Economy Class,4,4,4,3,1,2,0,3.0,Beijing,Xi'an
22,Air china,China,Business/Economy class CDG-PEK-PVG-CDG. CDG-PE...,2012-06-17,Solo Leisure,Business Class,3,4,2,2,2,3,1,3.0,Beijing,Xi'an
35,Air china,China,Madrid-Beijing A330 and Beijing-Frankfurt B747...,2012-08-21,Solo Leisure,Economy Class,3,3,4,1,3,4,1,3.0,Beijing,Xi'an
40,Air china,China,Business class London Gatwick to Beijing to Ul...,2012-09-04,Solo Leisure,Business Class,3,4,3,1,3,4,0,3.0,Beijing,Xi'an
47,Air china,China,PEK-FRA on a very old B744. Actually I chose A...,2012-09-11,Solo Leisure,Economy Class,3,4,3,1,3,4,0,3.0,Beijing,Xi'an
53,Air china,China,The price difference between Singapore Airline...,2012-11-09,Solo Leisure,Business Class,4,3,3,3,3,3,0,3.0,Beijing,Xi'an
61,Air china,China,Hong Kong to Beijing. Wanted to fly Hong Kong ...,2012-12-28,Solo Leisure,Business Class,3,4,3,3,3,4,0,3.0,Beijing,Xi'an


# Now Plotting the distribution of the numerical columns after imputing the missing values

fig , ax = plt.subplots(3,2,figsize=(15,15))

for i, subplot in zip(num_col, ax.flatten()):
    sns.distplot(df[i], ax=subplot)
plt.show()

# Plotting the distribution of the categorical columns after imputing the missing values

fig , ax = plt.subplots(3,2,figsize=(20,15))

for i, subplot in zip(cat_col, ax.flatten()):
    sns.countplot(x=i,data=df, ax=subplot, palette='CMRmap_r')
    if i in ['Airline','Country']:
        for label in subplot.get_xticklabels():
            label.set_rotation(90)
plt.show()

In [ ]:
df.to_csv('airline_df_cleaned.csv',index=False)